In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set()

In [2]:
df = pd.read_csv('udemy_data/1.02. Multiple linear regression.csv')
df.head()

,SAT,GPA,"Rand 1,2,3"
0,1714,2.40,1
1,1664,2.52,3
2,1760,2.54,3
3,1685,2.74,3
4,1693,2.83,2


In [3]:
y = df['GPA']
x = df.drop('GPA', axis=1) #drop used for delete the (axis 0 means rows and axis 1 means column)

In [4]:
#regression
reg = LinearRegression(normalize=True)
reg.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [5]:
#coef
reg.coef_ #firs for SAT and second for random 123

array([ 0.00165354, -0.00826982])

In [6]:
#intercept
reg.intercept_

0.29603261264909486

In [7]:
#summart
regres_summary = pd.DataFrame([['Bias'],['SAT'],['Rand 1,2,3']],columns=['Features'])
regres_summary['Weights'] = reg.intercept_,reg.coef_[0],reg.coef_[1] #intercept or bias are equal
regres_summary

,Features,Weights
0,Bias,0.296033
1,SAT,0.001654
2,"Rand 1,2,3",-0.008270


<b>Calculating R</b>

In [8]:
#R-square
reg.score(x,y)

0.4066811952814285

<b>Calculate Adj R-Square</b>

<b>Formula of Adj R-Square</b><br>
$R^2_{adj.} = 1 - (1-R^2)*\frac{n-1}{n-p-1}$

In [9]:
r2 = reg.score(x,y)
n = x.shape[0] #number of observation
p = x.shape[1] #number of feature
adj_r2 = 1-(1-r2)*((n-1)/(n-p-1))
adj_r2
#we can see that adj R-square < R-square, so it works correctly

0.39203134825134023

<b>Feature Selection</b>

In [10]:
from sklearn.feature_selection import f_regression

In [11]:
f_regression(x,y)
# 1 array is F-statistics and 2 array is P-Values

(array([56.04804786,  0.17558437]), array([7.19951844e-11, 6.76291372e-01]))

In [12]:
#we just need the P-values to see the feature
p_value = f_regression(x,y)[1]
p_value

array([7.19951844e-11, 6.76291372e-01])

In [13]:
p_value.round(3) #SAT, random 123

array([0.   , 0.676])

we can see that p-value SAT = 0 and random 123 = 0.676
the conclusion is SAT is usefull and random 123 useless

<b>Create Summary Table</b>

In [14]:
reg_summary = pd.DataFrame(data = x.columns.values, columns=['Features'])
reg_summary

,Features
0,SAT
1,"Rand 1,2,3"


In [15]:
reg_summary['Coefficients'] = reg.coef_
reg_summary['P-value']= p_value.round(3)
reg_summary

,Features,Coefficients,P-value
0,SAT,0.001654,0.000
1,"Rand 1,2,3",-0.008270,0.676


In [16]:
#make a new data and try to predict that data
new_data = {'SAT':[1800,1750],'Random 1,2,3':[1,2]}
new_data = pd.DataFrame(new_data)
new_data

,SAT,"Random 1,2,3"
0,1800,1
1,1750,2


In [17]:
#predict result
reg.predict(new_data)

array([3.26413803, 3.17319112])

<b>Standardization</b>
$std = \frac{x-miu}{stdv}$

In [18]:
#standardization is used for make the data normaly distributed, se the value between -1 and 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [19]:
scaler.fit(x)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [20]:
x_scaled = scaler.transform(x)

<b>Regression with Scale Feature</b>

In [21]:
reg = LinearRegression()
reg.fit(x_scaled, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [22]:
reg.coef_

array([ 0.17181389, -0.00703007])

In [23]:
reg.intercept_

3.330238095238095

<b>Create Summary Table</b>

In [24]:
reg_summary = pd.DataFrame([['Bias'],['SAT'],['Rand 1,2,3']], columns=['Features'])

In [25]:
reg_summary

,Features
0,Bias
1,SAT
2,"Rand 1,2,3"


In [26]:
reg_summary['Weights']=reg.intercept_, reg.coef_[0], reg.coef_[1]
reg_summary
#closed with 0 give small impact

,Features,Weights
0,Bias,3.330238
1,SAT,0.171814
2,"Rand 1,2,3",-0.007030


<b>Predict GPA with scaling</b>

In [27]:
#new data that want to predict with scaler model
new_data = {'SAT':[1800,1750],'Random 1,2,3':[1,2]}
new_data = pd.DataFrame(new_data)
new_data

,SAT,"Random 1,2,3"
0,1800,1
1,1750,2


In [28]:
reg.predict(new_data)
#it is invalid, so we also need to scaling that data

array([312.58821497, 303.99049026])

In [29]:
new_data_scale = scaler.transform(new_data)
new_data_scale

array([[-0.43571643, -1.24637147],
       [-0.91691786, -0.07002087]])

In [30]:
reg.predict(new_data_scale)

array([3.26413803, 3.17319112])

<b>what if we remove random 1,2,3</b>

In [31]:
reg_simple = LinearRegression()

In [32]:
#take just SAT value
# z = x_scaled[:,0].reshape(-1,1)
# z.shape
reg_simple.fit(x_scaled[:,0].reshape(-1,1), y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [33]:
reg_simple.predict(new_data_scale[:,0].reshape(-1,1))

array([3.25527879, 3.17249439])

- If we round 2 digit behind the dot, we got the same value 3.26 and 3.17
- This is why in sklearn p-value isn't needed
- The feature scaling not affect to final result